#### Notebook for running Chain-of-Thought with supporting context experiments 

In [3]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-o5678901234567890'

In [4]:
import sys, os
sys.path.append('..')
root = '../root/'

In [5]:
import joblib
import numpy as np
from agents import CoTAgent, ReflexionStrategy
from util import summarize_trial, log_trial, save_agents

() {'temperature': 0, 'max_tokens': 250, 'stop': ['\n']} {'temperature': 0, 'max_tokens': 250, 'stop': ['\n']}
() {'temperature': 0, 'max_tokens': 250, 'stop': ['\n']} {'temperature': 0, 'max_tokens': 250, 'stop': ['\n']}
() {'temperature': 0, 'max_tokens': 100, 'stop': ['\n']} {'temperature': 0, 'max_tokens': 100, 'stop': ['\n']}
() {'temperature': 0, 'max_tokens': 100, 'stop': ['\n']} {'temperature': 0, 'max_tokens': 100, 'stop': ['\n']}
() {'temperature': 0, 'max_tokens': 250} {'temperature': 0, 'max_tokens': 250}


#### Load the HotPotQA Sample

In [6]:
hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

hotpot['supporting_paragraphs'] = None
for ind, row in hotpot.iterrows():
    supporting_articles = row['supporting_facts']['title']
    articles = row['context']['title']
    sentences = row['context']['sentences'] 
    supporting_paragraphs = []
    for article in supporting_articles:
        supporting_paragraph = ''.join(sentences[np.where(articles == article)][0])
        supporting_paragraphs.append(supporting_paragraph)
    supporting_paragraphs = '\n\n'.join(supporting_paragraphs)
    hotpot.at[ind, 'supporting_paragraphs'] = supporting_paragraphs

#### Define the Reflexion Strategy

In [7]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [8]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

#### Initialize a CoTAgent for each question

In [9]:
from prompts import cot_agent_prompt, cot_reflect_agent_prompt, cot_reflect_prompt
from fewshots import COT, COT_REFLECT
agents = [CoTAgent(row['question'],
                   row['supporting_paragraphs'],
                   row['answer'],
                   agent_prompt=cot_agent_prompt if strategy == ReflexionStrategy.NONE else cot_reflect_agent_prompt,
                   cot_examples=COT,
                   reflect_prompt=cot_reflect_prompt,
                   reflect_examples=COT_REFLECT,
                    ) for _, row in hotpot.iterrows()]

#### Run `n` trials

In [10]:
n = 5
trial = 0
log = ''

In [11]:
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        agent.run(reflexion_strategy = strategy)
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_trial(agents, trial)
    correct, incorrect = summarize_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

> prompt(chat): Solve a question answering task by having a Thought, then Finish with your answer. Thought can reason about the current situation. Finish[answer] returns the answer and finishes the task. You will be given context that you should use to help you answer the question.
Here are some examples:
Relevant Context: The Nile River is the longest river in the world, spanning approximately 6,650 kilometers (4,132 miles) in length. It flows through eleven countries in northeastern Africa, including Egypt, Sudan, and Uganda.
Question: What is the longest river in the world?
Thought: The question asks for the longest river in the world, which I know is the Nile River based on the context provided.
Action: Finish[Nile River]

Relevant Context: Ludwig van Beethoven was a German composer and pianist who was a crucial figure in the transition between the Classical and Romantic eras in Western classical music. One of his most famous compositions is the Symphony No. 9, also known as the "C

#### Save the result log

In [12]:
with open(os.path.join(root, 'CoT', 'context', strategy.value, f'{len(agents)}_questions_{trial}_trials.txt'), 'w') as f:
    f.write(log)
save_agents(agents, os.path.join(root, 'CoT', 'context', strategy.value, 'agents'))

TypeError: cannot pickle '_thread.RLock' object